# Objektsorientering  

## Inheritance

In [ ]:
class Square: 

    def __init__(self, side):
        self._side = side

    @property
    def side(self, value):
        return self._side

    @property
    def area(self):
        return self._side ** 2

    
class Rectangle:

    def __init__(self, long, short):
        self._long = long
        self._short = short
    
    @property
    def area(self):
        return self._long * self._short

s = Square(2)
r = Rectangle(2, 4)

s.area, r.area

In [ ]:
from abc import ABC, abstractmethod

class Shape(ABC):

    @abstractmethod
    def area(self):
        pass


class Square: 

    def __init__(self, side):
        self._side = side

    @property
    def side(self, value):
        return self._side

    @property
    def area(self):
        return self._side ** 2

    
class Rectangle(Shape):

    def __init__(self, long, short):
        self._long = long
        self._short = short
    
    @property
    def area(self):
        return self._long * self._short

s = Square(2)
r = Rectangle(2, 4)
shape = Shape()
s.area

När vi använder abstrakta metoder i Python är det bara en decorator (inte abstrakt) som säger att vi inte får instansiera det. 

In [ ]:
class Person:
    def __init__(self, name):
        self._name = name


class Student(Person):
    @property
    def is_studying(self):
        print(f"{self._name} is studying")
        return True


class Unemployed(Student):
    @property
    def is_studying(self):
        print(f"{self._name} is not studying")
        return False

    @property
    def is_working(self):
        print(f"{self._name} is not unemployed")
        return False


u = Unemployed("Ada")
s = Student("Einstein")
p = Person("Church")

s._name, u._name, p._name

En god rekommendation är att inte ärva från flera klasser. 

In [ ]:
class WithMessage(object):
    def __init__(self, message):
        self.message = message
    
    def __enter__(self):
        print(f"Entering with message {self.message}")
        return self.message
    
    def __exit__(self, *args):
        print("Exiting with message")


with WithMessage("Hello") as msg:           # msg namnger det som returneras från __enter__
    print(msg)



In [ ]:
import matplotlib.pyplot as plt


class Vector:           # vector är som matematiska listor
    """A class to represent Euclidean vectors"""

    def __init__(self, *numbers):         # *numbers är som en dict med alla följande parametrar i
        # error checking
        for number in numbers:
            if not isinstance(number, (float, int)):
                raise TypeError(f"{number} is not a valid number")
        if len(numbers) <= 0:
            raise ValueError("Vectors can't be empty")
        
        self._numbers = tuple(float(number) for number in numbers)

    @property
    def numbers(self):
        return self._numbers
    
    @staticmethod
    def validate2d(instance):
        if not len(instance) == 2:
            raise ValueError("The vector is not 2D")
        return True
    
    def __add__(self, other):
        if self.validate_vector(other):
            numbers = (a + b for a, b in zip(self.numbers, other.numbers))
            return Vector(*numbers)
        
    def __sub__(self, other):
        if self.validate_vector(other):
            numbers = (a - b for a, b in zip(self.numbers, other.numbers))
            return Vector(*numbers)
    
    def __mul__(self, value):                       # här kan vi bara multiplicera t ex u * 3
        if not isinstance(value, (int, float)):
            raise TypeError("Value must be a scalar (int or float).")
        numbers = (a * value for a in self.numbers)
        return Vector(*numbers)
    
    def __rmul__(self, value):                      # detta gör vi för att kunna multiplicera t ex 3 * u
        return self * value

    def __len__(self):
        return len(self.numbers)

    def validate_vector(self, other):
        if not isinstance(other, Vector): 
            raise ValueError(f"{other} is not a Vector")
        if len(self) != len(other):
            raise TypeError(f"Dimensions of vectors not equal: {len(other)} != {len(self)}")
        return True
    
    def __abs__(self):
        """Returns the Euclidean norm of the vector, aka the L2-norm"""   # kallas även manhattan distance ??? enligt Raphael
        return sum(a ** 2 for a in self.numbers) ** 0.5                   # ** 0.5 är samma sak som att ta kvadratroten ur

    def __repr__(self):
        return f"Vector{self.numbers}"
    
    def __getitem__(self, index):
        return self.numbers[index]
    
    @staticmethod
    def plot(*vectors):
        X, Y = [], []
        for v in vectors:
            if Vector.validate2d(v):
                X.append(v[0])
                Y.append(v[1])
        
        originX = originY = tuple(0 for _ in range(len(X)))
        plt.quiver(originX, originY, X, Y, angles = "xy", scale_units = "xy", scale = 1)
        plt.xlim(-2, 10)
        plt.ylim(-2, 10)
        plt.grid()
        plt.show()


v = Vector(2, 4)
u = Vector(4, 5)
print(u + v, u - v, u * 3)         # när det står + så kallar den på __add__, u blir self och v blir other

# multiplikation är kommutativt (spelar ingen roll vilket objekt som står till höger eller vänster) men multiplikation med en vektor/scalar (?) är inte det

abs(-1)         # abs är en inbyggd funktion som returnerar absolutbeloppet av ett tal
print(abs(-1))

# inom matematik skriver vi absolutbelopp som |n|
# |n| = n if n > 0
# |n| = -n if n < 0       (det är alltså minus minus n, alltså positivt därefter)
# | x - a |               alla punkter vars avstånd till a är x. den hanterar både positiva och negativa tal
# absolutbelopp på vector är hur lång den är, Euclidiskt avstånd

print(abs(v))
print(u[1])
print(Vector.validate2d(u))
Vector.plot(v, u, u+v, u-v)

En viktig sak med linjär algebra:
- ta norm
- ta magnitudvärde
det betyder avståndet från origo

# Testfall

In [ ]:
import unittest 

class TestVector(unittest.TestCase):
    def set_up(self):
        self.x, self.y = 1, 2
    
    def create_2D_vector(self):
        return Vector(self.x, self.y)
    
    def test_create_2D_vector(self):
        v = self.create_2D_vector()
        self.assertEqual(v.numbers, (self.x, self.y))       # är de inte lika så får vi fel och det kraschar

    def test_create_5D_vector(self):
        v = Vector(-1, 0, -5, -50.2, 52.2)
        self.assertEqual(v.numbers, (-1, 0, -5, -50.2, 52.2))
    
    def test_create_empty_vector(self):
        with self.assertRaises(ValueError):
            v = Vector()
    
    def test_2_vectors_equal(self):                         # ger fel för vi i koden inte jämför vektorer
        v1 = self.create_2D_vector
        v2 = Vector(1, 2)
        self.assertEqual(v1, v2)

    def test_2_vectors_not_equal(self):
        v1 = self.create_2D_vector
        v2 = Vector(3, 2)
        self.assertNotEqual(v1, v2)

    def test_add_2_vectors(self):
        v1 = self.create_2D_vector
        v2 = Vector(1, -2)
        self.assertEqual(v1 + v22, Vector(2, 0))

    def test_add_vector_diff_dimensions(self):
        v1 = self.create_2D_vector
        v2 = Vector(1, 0, 1)
        with self.assertRaises(TypeError):
            _ = v1 + v2                                     # om någon skriver så ska vi inte använda denna variabel, don't care
    
    def test_multiply_scalar(self):
        v1 = self.create_2D_vector
        v2 = v1 * 5
        self.assertEqual(v2, Vector(5, 10))

    def test_rmultiply_scalar(self):
        v1 = self.create_2D_vector
        v2 = 5 * v1
        self.assertEqual(v2, Vector(5, 10))



if __name__ == "__main__":
    unittest.main()